In [5]:
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen
import requests
import pandas as pd
import re
import time
import os

In [70]:
def Crawler(x_code, x_page):
    # start = time.time()
    df = pd.DataFrame(columns = [
    'date',
    'title',
    'content',
    'click',
    'good',
    'bad'
    ])

    if page == 'all':
        # 마지막 페이지
        url = "https://finance.naver.com/item/board.nhn?code={}&page={}".format(x_code, 1)
        html = requests.get(url, headers = headers).text
        soup = bs(html, 'html.parser')
        last_page = soup.select('#content > div.section.inner_sub > table:nth-child(3) > tbody > tr > td:nth-child(2) > table > tbody > tr > td.pgRR ')[0]
        last_page = last_page.find('a')
        last_page = last_page.attrs['href'][33:]
        last_page = int(last_page)
        # print(last_page)
        
        for p in range(1, last_page+1):
            url = "https://finance.naver.com/item/board.nhn?code={}&page={}".format(x_code, p)
            html = requests.get(url, headers = headers).text
            soup = bs(html, 'html.parser')
            base = soup.select('#content > div.section.inner_sub > table.type2 > tbody')[0]
            
            for i in range(3, 26):
                    # 5칸마다 있는 줄에 해당할 경우 contunue
                try:
                    if i == 8 or i == 14 or i == 20:
                        continue

                    date = base.select('tr:nth-child({}) > td:nth-child(1) > span'.format(i))[0].text
                except:
                    break

                title = base.select('tr:nth-child({}) > td.title > a'.format(i))[0].text
                click = base.select('tr:nth-child({}) > td:nth-child(4) > span'.format(i))[0].text
                good = base.select('tr:nth-child({}) > td:nth-child(5) > strong'.format(i))[0].text
                bad = base.select('tr:nth-child({}) > td:nth-child(6) > strong'.format(i))[0].text

                # href 얻기
                href = base.select('tr:nth-child({}) > td.title'.format(i))[0]
                href = href.find('a')
                href = href.attrs['href']

                # 본문 내용 얻기
                content_url = ('https://finance.naver.com'+ href)
                content_html = requests.get(content_url, headers = headers).text
                content_soup = bs(content_html, 'html.parser')
                content = content_soup.find("div", "view_se").text

                # df에 저장
                result = {
                    'date':date,
                    'title':title,
                    'content':content,
                    'click':click,
                    'good':good,
                    'bad':bad
                    }
                df = df.append(result, ignore_index = True)
                #end = time.time()
                #print("경과 시간: {}".format(end - start))
    else:
        for p in range(1, x_page+1):
            url = "https://finance.naver.com/item/board.nhn?code={}&page={}".format(x_code, p)
            html = requests.get(url, headers = headers).text
            soup = bs(html, 'html.parser')
            base = soup.select('#content > div.section.inner_sub > table.type2 > tbody')[0]

            for i in range(3, 26):
                # 5칸마다 있는 줄에 해당할 경우 contunue
                try:
                    if i == 8 or i == 14 or i == 20:
                        continue

                    date = base.select('tr:nth-child({}) > td:nth-child(1) > span'.format(i))[0].text
                except:
                    break

                title = base.select('tr:nth-child({}) > td.title > a'.format(i))[0].text
                click = base.select('tr:nth-child({}) > td:nth-child(4) > span'.format(i))[0].text
                good = base.select('tr:nth-child({}) > td:nth-child(5) > strong'.format(i))[0].text
                bad = base.select('tr:nth-child({}) > td:nth-child(6) > strong'.format(i))[0].text

                # href 얻기
                href = base.select('tr:nth-child({}) > td.title'.format(i))[0]
                href = href.find('a')
                href = href.attrs['href']

                # 본문 내용 얻기
                content_url = ('https://finance.naver.com'+ href)
                content_html = requests.get(content_url, headers = headers).text
                content_soup = bs(content_html, 'html.parser')
                content = content_soup.find("div", "view_se").text

                # df에 저장
                result = {
                    'date':date,
                    'title':title,
                    'content':content,
                    'click':click,
                    'good':good,
                    'bad':bad
                    }
                df = df.append(result, ignore_index = True)
            #end = time.time()
            #print("경과 시간: {}".format(end - start))
    return df

In [71]:
# Hyper-parameters
code = '094170'
page = 10# 마지막 페이지까지 하고 싶으면 'all'
headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3)'}

In [72]:
#Crawling
df = Crawler(code, page)

,date,title,content,click,good,bad
0,2021.02.18 17:17,이거 사고 싶은데 내일 떨어질확률이 높을...\n\t\t\t\t\n\t\t\t\t,제가 생각없다고 생각하시는 어른분들은 제가 만으로 14살인거 생각해주세요,10,0,0
1,2021.02.18 15:25,10300에 대기\n\t\t\t\t\n\t\t\t\t,빨리 던져줘,41,0,0
2,2021.02.18 13:25,C8\n\t\t\t\t\n\t\t\t\t,또 또 짜증나게 하는구만!\rㅎ ㅎ ㅎ,87,0,0
3,2021.02.18 09:20,그냥 개미털기임\n\t\t\t\t\n\t\t\t\t,천천히보유\r 올해 기대주,133,0,0
4,2021.02.18 11:19,그냥 개미털기임\n\t\t\t\t\n\t\t\t\t,"10,500까지는 기다려야지.. 지금 달라들면 털려유\r밑에다 받쳐놓고 대차물량 던...",82,0,0
...,...,...,...,...,...,...
195,2020.12.27 08:51,음...... 올해는 회사 보너스 주시려...\n\t\t\t\t[1]\n,열심히 일 잘하고 있는데.\r 보너스는 주시겠죠?\r 회사 주가도 많이 올라갔는데....,541,1,1
196,2020.12.25 23:19,(倭색풍의 가녀린 간드러진)《기름기 쩐 ...\n\t\t\t\t[1]\n,<들어가기 전에> (가필본) \r 0 지저분한 사생활 여부 등에 관해서는 여기서는...,523,2,3
197,2020.12.25 09:15,4700원대에 들어와서\n\t\t\t\t[1]\n,7천원에 팔고 갔는데 12000원 되어있?ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ와~\r만원아나텍...,767,4,1
198,2020.12.25 00:09,변종\n\t\t\t\t[1]\n,https://youtu.be/KgM5pv1JHjc,494,0,0


In [6]:
# 결과 저장
df.to_excel('{}_result.xlsx'.format(code))